# 🫁 Analyse Complète du Dataset Chest X-Ray Pneumonia

## 📊 Crédits et Source des Données

**Dataset :** [Chest X-Ray Images (Pneumonia)](https://www.kaggle.com/datasets/paultimothymooney/chest-xray-pneumonia)
**Source :** Kaggle
**Créateur :** Paul Mooney
**Auteur de l'analyse :** Dady Akrou Cyrille

---

## 🎯 Objectif du Projet

Ce notebook présente une analyse complète du dataset de radiographies thoraciques pour la détection automatique de pneumonie. L'objectif est de développer un modèle de deep learning capable de classifier les radiographies en deux catégories :

- **NORMAL** : Radiographies sans pneumonie
- **PNEUMONIA** : Radiographies avec pneumonie

## 📋 Table des Matières

1. [Configuration et Imports](#1-configuration-et-imports)
2. [Exploration du Dataset](#2-exploration-du-dataset)
3. [Analyse Statistique](#3-analyse-statistique)
4. [Visualisations Avancées](#4-visualisations-avancées)
5. [Analyse des Images](#5-analyse-des-images)
6. [Recommandations ML](#6-recommandations-ml)
7. [Conclusions](#7-conclusions)

## 1. Configuration et Imports

### Installation des dépendances

Si vous exécutez ce notebook pour la première fois, décommentez et exécutez la cellule suivante :

In [ ]:
# !pip install -r requirements.txt

### Imports et configuration

In [ ]:
# Imports standards
import os
import sys
import json
import warnings
from pathlib import Path

# Imports pour l'analyse de données
import pandas as pd
import numpy as np

# Imports pour la visualisation
import matplotlib.pyplot as plt
import seaborn as sns
from PIL import Image

# Imports pour le traitement d'images
import cv2

# Configuration
warnings.filterwarnings('ignore')
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

# Configuration matplotlib pour l'affichage en français
plt.rcParams['font.size'] = 12
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['axes.grid'] = True
plt.rcParams['grid.alpha'] = 0.3

print("✅ Configuration terminée avec succès !")

### Chargement des modules du projet

In [ ]:
# Chargement des modules personnalisés
from config import *
from utils import *

# Affichage de l'en-tête du projet
afficher_entete_projet()

print(f"📁 Répertoire de travail : {os.getcwd()}")
print(f"📊 Dataset configuré : {DATASET_PATH}")

## 2. Exploration du Dataset

### Validation de la structure du dataset

In [ ]:
# Validation de la structure
print("🔍 Validation de la structure du dataset...")

structure_valide = valider_structure_dataset(DATASET_PATH)

if structure_valide:
    print("✅ Structure du dataset validée avec succès !")
else:
    print("❌ Problème détecté dans la structure du dataset")
    print("Veuillez vérifier que le dataset est correctement organisé.")

In [ ]:
# Exploration de la structure des dossiers
print("📂 Structure du dataset :")
print("=" * 50)

for subset in SUBSETS:
    subset_path = os.path.join(DATASET_PATH, subset)
    if os.path.exists(subset_path):
        print(f"\n📁 {subset.upper()}/")
        for classe in CLASSES:
            classe_path = os.path.join(subset_path, classe)
            if os.path.exists(classe_path):
                nb_images = compter_images(classe_path)
                print(f"  └── {classe}: {nb_images:,} images")
            else:
                print(f"  └── {classe}: ❌ Dossier manquant")
    else:
        print(f"\n❌ {subset.upper()}/: Dossier manquant")

## 3. Analyse Statistique

### Statistiques générales du dataset

In [ ]:
# Obtention des statistiques complètes
print("📊 Calcul des statistiques du dataset...")

stats = obtenir_statistiques_dataset(DATASET_PATH)

# Affichage des statistiques générales
print("\n🔢 STATISTIQUES GÉNÉRALES")
print("=" * 50)
print(f"📊 Total d'images : {stats['total_images']:,}")
print(f"📁 Nombre de classes : {len(CLASSES)}")
print(f"📂 Nombre de sous-ensembles : {len(SUBSETS)}")

# Statistiques par sous-ensemble
print("\n📂 RÉPARTITION PAR SOUS-ENSEMBLE")
print("=" * 50)

for subset in SUBSETS:
    if subset in stats['par_subset']:
        subset_stats = stats['par_subset'][subset]
        total_subset = sum(subset_stats.values())
        pourcentage = (total_subset / stats['total_images']) * 100
        
        print(f"\n📁 {subset.upper()}:")
        print(f"   Total: {total_subset:,} images ({pourcentage:.1f}%)")
        
        for classe in CLASSES:
            if classe in subset_stats:
                nb_images = subset_stats[classe]
                pourcentage_classe = (nb_images / total_subset) * 100 if total_subset > 0 else 0
                print(f"   └── {classe}: {nb_images:,} ({pourcentage_classe:.1f}%)")
            else:
                print(f"   └── {classe}: 0 (0.0%)")
    else:
        print(f"\n❌ {subset.upper()}: Aucune donnée trouvée")

### Analyse du déséquilibre des classes

In [ ]:
# Calcul des poids de classes
print("⚖️ ANALYSE DU DÉSÉQUILIBRE DES CLASSES")
print("=" * 50)

poids_classes = calculer_poids_classes(stats)

# Statistiques globales par classe
total_par_classe = {}
for classe in CLASSES:
    total = 0
    for subset in SUBSETS:
        if subset in stats['par_subset'] and classe in stats['par_subset'][subset]:
            total += stats['par_subset'][subset][classe]
    total_par_classe[classe] = total

print("📊 Répartition globale par classe :")
for classe in CLASSES:
    total = total_par_classe[classe]
    pourcentage = (total / stats['total_images']) * 100
    print(f"   {classe}: {total:,} images ({pourcentage:.1f}%)")

# Calcul du ratio de déséquilibre
if len(total_par_classe) == 2:
    classes_list = list(total_par_classe.keys())
    ratio = max(total_par_classe.values()) / min(total_par_classe.values())
    classe_majoritaire = max(total_par_classe, key=total_par_classe.get)
    classe_minoritaire = min(total_par_classe, key=total_par_classe.get)
    
    print(f"\n⚖️ Ratio de déséquilibre : {ratio:.2f}:1")
    print(f"   Classe majoritaire : {classe_majoritaire}")
    print(f"   Classe minoritaire : {classe_minoritaire}")

print("\n🎯 Poids recommandés pour l'entraînement :")
for classe, poids in poids_classes.items():
    print(f"   {classe}: {poids:.4f}")

## 4. Visualisations Avancées

### Graphiques de distribution

In [ ]:
# Création des visualisations
print("📈 Génération des visualisations...")

# Préparation des données pour les graphiques
data_viz = []

for subset in SUBSETS:
    if subset in stats['par_subset']:
        for classe, count in stats['par_subset'][subset].items():
            data_viz.append({
                'Sous-ensemble': subset.capitalize(),
                'Classe': classe,
                'Nombre': count
            })

df_viz = pd.DataFrame(data_viz)

# Graphique 1: Distribution par sous-ensemble et classe
fig, axes = plt.subplots(2, 2, figsize=(16, 12))
fig.suptitle('📊 Analyse Complète du Dataset Chest X-Ray Pneumonia', fontsize=16, fontweight='bold')

# Graphique en barres groupées
ax1 = axes[0, 0]
df_pivot = df_viz.pivot(index='Sous-ensemble', columns='Classe', values='Nombre')
df_pivot.plot(kind='bar', ax=ax1, color=['skyblue', 'lightcoral'])
ax1.set_title('Distribution par Sous-ensemble et Classe')
ax1.set_xlabel('Sous-ensemble')
ax1.set_ylabel('Nombre d\'images')
ax1.legend(title='Classe')
ax1.tick_params(axis='x', rotation=45)

# Graphique en secteurs pour la distribution globale
ax2 = axes[0, 1]
total_par_classe_list = [total_par_classe[classe] for classe in CLASSES]
colors = ['lightblue', 'lightcoral']
wedges, texts, autotexts = ax2.pie(total_par_classe_list, labels=CLASSES, autopct='%1.1f%%', 
                                   colors=colors, startangle=90)
ax2.set_title('Répartition Globale des Classes')

# Graphique en barres horizontales pour les sous-ensembles
ax3 = axes[1, 0]
subset_totals = [sum(stats['par_subset'][subset].values()) if subset in stats['par_subset'] else 0 
                 for subset in SUBSETS]
bars = ax3.barh(SUBSETS, subset_totals, color=['lightgreen', 'orange', 'lightpink'])
ax3.set_title('Nombre d\'Images par Sous-ensemble')
ax3.set_xlabel('Nombre d\'images')

# Ajout des valeurs sur les barres
for i, bar in enumerate(bars):
    width = bar.get_width()
    ax3.text(width + 50, bar.get_y() + bar.get_height()/2, 
             f'{int(width):,}', ha='left', va='center')

# Graphique de comparaison des ratios
ax4 = axes[1, 1]
ratios_data = []
for subset in SUBSETS:
    if subset in stats['par_subset']:
        subset_stats = stats['par_subset'][subset]
        if len(subset_stats) == 2:
            ratio = max(subset_stats.values()) / min(subset_stats.values())
            ratios_data.append(ratio)
        else:
            ratios_data.append(0)
    else:
        ratios_data.append(0)

bars = ax4.bar(SUBSETS, ratios_data, color=['lightblue', 'lightgreen', 'lightcoral'])
ax4.set_title('Ratio de Déséquilibre par Sous-ensemble')
ax4.set_ylabel('Ratio (Majoritaire:Minoritaire)')
ax4.tick_params(axis='x', rotation=45)

# Ajout d'une ligne de référence pour un ratio équilibré
ax4.axhline(y=1, color='red', linestyle='--', alpha=0.7, label='Équilibre parfait')
ax4.legend()

# Ajout des valeurs sur les barres
for i, bar in enumerate(bars):
    height = bar.get_height()
    if height > 0:
        ax4.text(bar.get_x() + bar.get_width()/2, height + 0.1, 
                 f'{height:.2f}:1', ha='center', va='bottom')

plt.tight_layout()
plt.show()

print("✅ Visualisations générées avec succès !")

## 5. Analyse des Images

### Propriétés des images (dimensions, formats, tailles)

In [ ]:
# Analyse des propriétés des images
print("🖼️ Analyse des propriétés des images...")
print("⏳ Cette opération peut prendre quelques minutes...\n")

proprietes = analyser_proprietes_images(DATASET_PATH)

print("📊 PROPRIÉTÉS DES IMAGES")
print("=" * 50)

# Formats d'images
print("🎨 Formats détectés :")
for format_img, count in proprietes['formats'].items():
    pourcentage = (count / proprietes['total_analysees']) * 100
    print(f"   {format_img}: {count:,} images ({pourcentage:.1f}%)")

# Dimensions
print("\n📐 Dimensions des images :")
print(f"   Largeur moyenne : {proprietes['largeur_moyenne']:.0f} pixels")
print(f"   Hauteur moyenne : {proprietes['hauteur_moyenne']:.0f} pixels")
print(f"   Dimension la plus fréquente : {proprietes['dimension_plus_frequente']}")

# Tailles de fichiers
print("\n💾 Tailles des fichiers :")
print(f"   Taille moyenne : {proprietes['taille_moyenne_mb']:.2f} MB")
print(f"   Taille médiane : {proprietes['taille_mediane_mb']:.2f} MB")
print(f"   Taille minimale : {proprietes['taille_min_mb']:.3f} MB")
print(f"   Taille maximale : {proprietes['taille_max_mb']:.2f} MB")

print(f"\n🔍 Images analysées : {proprietes['total_analysees']:,} / {stats['total_images']:,}")

### Visualisation des propriétés des images

In [ ]:
# Visualisation des propriétés des images
fig, axes = plt.subplots(2, 2, figsize=(16, 12))
fig.suptitle('🖼️ Analyse des Propriétés des Images', fontsize=16, fontweight='bold')

# Graphique 1: Distribution des largeurs
ax1 = axes[0, 0]
largeurs = [dim[0] for dim in proprietes['dimensions']]
ax1.hist(largeurs, bins=30, color='skyblue', alpha=0.7, edgecolor='black')
ax1.set_title('Distribution des Largeurs')
ax1.set_xlabel('Largeur (pixels)')
ax1.set_ylabel('Fréquence')
ax1.axvline(proprietes['largeur_moyenne'], color='red', linestyle='--', 
           label=f'Moyenne: {proprietes["largeur_moyenne"]:.0f}px')
ax1.legend()

# Graphique 2: Distribution des hauteurs
ax2 = axes[0, 1]
hauteurs = [dim[1] for dim in proprietes['dimensions']]
ax2.hist(hauteurs, bins=30, color='lightcoral', alpha=0.7, edgecolor='black')
ax2.set_title('Distribution des Hauteurs')
ax2.set_xlabel('Hauteur (pixels)')
ax2.set_ylabel('Fréquence')
ax2.axvline(proprietes['hauteur_moyenne'], color='red', linestyle='--', 
           label=f'Moyenne: {proprietes["hauteur_moyenne"]:.0f}px')
ax2.legend()

# Graphique 3: Distribution des tailles de fichiers
ax3 = axes[1, 0]
tailles_mb = [taille / (1024*1024) for taille in proprietes['tailles']]
ax3.hist(tailles_mb, bins=30, color='lightgreen', alpha=0.7, edgecolor='black')
ax3.set_title('Distribution des Tailles de Fichiers')
ax3.set_xlabel('Taille (MB)')
ax3.set_ylabel('Fréquence')
ax3.axvline(proprietes['taille_moyenne_mb'], color='red', linestyle='--', 
           label=f'Moyenne: {proprietes["taille_moyenne_mb"]:.2f}MB')
ax3.legend()

# Graphique 4: Scatter plot largeur vs hauteur
ax4 = axes[1, 1]
ax4.scatter(largeurs, hauteurs, alpha=0.6, color='purple', s=10)
ax4.set_title('Relation Largeur vs Hauteur')
ax4.set_xlabel('Largeur (pixels)')
ax4.set_ylabel('Hauteur (pixels)')

# Ajout de la ligne de ratio 1:1
max_dim = max(max(largeurs), max(hauteurs))
ax4.plot([0, max_dim], [0, max_dim], 'r--', alpha=0.5, label='Ratio 1:1')
ax4.legend()

plt.tight_layout()
plt.show()

print("✅ Visualisations des propriétés générées avec succès !")

### Échantillons d'images par classe

In [ ]:
# Affichage d'échantillons d'images
print("🖼️ Affichage d'échantillons d'images par classe...")

def afficher_echantillons(dataset_path, nb_echantillons=3):
    """Affiche des échantillons d'images pour chaque classe"""
    
    fig, axes = plt.subplots(len(CLASSES), nb_echantillons, 
                            figsize=(nb_echantillons * 4, len(CLASSES) * 4))
    
    if len(CLASSES) == 1:
        axes = [axes]
    
    fig.suptitle('🖼️ Échantillons d\'Images par Classe', fontsize=16, fontweight='bold')
    
    for i, classe in enumerate(CLASSES):
        # Chercher des images dans le dossier train en priorité
        classe_path = os.path.join(dataset_path, 'train', classe)
        
        if not os.path.exists(classe_path):
            # Si pas de dossier train, chercher dans test
            classe_path = os.path.join(dataset_path, 'test', classe)
        
        if os.path.exists(classe_path):
            images = [f for f in os.listdir(classe_path) 
                     if f.lower().endswith(('.png', '.jpg', '.jpeg'))]
            
            # Sélectionner des échantillons aléatoires
            echantillons = np.random.choice(images, 
                                          min(nb_echantillons, len(images)), 
                                          replace=False)
            
            for j, image_name in enumerate(echantillons):
                image_path = os.path.join(classe_path, image_name)
                
                try:
                    # Charger et afficher l'image
                    img = Image.open(image_path)
                    
                    if len(CLASSES) == 1:
                        ax = axes[j]
                    else:
                        ax = axes[i, j]
                    
                    ax.imshow(img, cmap='gray' if img.mode == 'L' else None)
                    ax.set_title(f'{classe}\n{img.size[0]}x{img.size[1]}px')
                    ax.axis('off')
                    
                except Exception as e:
                    if len(CLASSES) == 1:
                        ax = axes[j]
                    else:
                        ax = axes[i, j]
                    
                    ax.text(0.5, 0.5, f'Erreur\nchargement\n{str(e)[:20]}...', 
                           ha='center', va='center', transform=ax.transAxes)
                    ax.axis('off')
        else:
            # Si le dossier n'existe pas
            for j in range(nb_echantillons):
                if len(CLASSES) == 1:
                    ax = axes[j]
                else:
                    ax = axes[i, j]
                
                ax.text(0.5, 0.5, f'Dossier\n{classe}\nintrouvable', 
                       ha='center', va='center', transform=ax.transAxes)
                ax.axis('off')
    
    plt.tight_layout()
    plt.show()

# Affichage des échantillons
afficher_echantillons(DATASET_PATH, nb_echantillons=4)

print("✅ Échantillons d'images affichés avec succès !")

## 6. Recommandations ML

### Stratégies recommandées pour ce dataset

In [ ]:
# Génération des recommandations ML
print("🎯 RECOMMANDATIONS MACHINE LEARNING")
print("=" * 60)

# Analyse du déséquilibre
ratio_desequilibre = max(total_par_classe.values()) / min(total_par_classe.values())

print("⚖️ GESTION DU DÉSÉQUILIBRE DES CLASSES")
print(f"   Ratio actuel : {ratio_desequilibre:.2f}:1")

if ratio_desequilibre > 2.0:
    print("   🚨 DÉSÉQUILIBRE SIGNIFICATIF DÉTECTÉ")
    print("   
   Stratégies recommandées :")
    print("   • Utiliser les poids de classes calculés")
    print("   • Appliquer des techniques de sur-échantillonnage (SMOTE)")
    print("   • Augmentation de données ciblée sur la classe minoritaire")
    print("   • Utiliser des métriques équilibrées (F1-score, AUC-ROC)")
elif ratio_desequilibre > 1.5:
    print("   ⚠️ Déséquilibre modéré")
    print("   Stratégies recommandées :")
    print("   • Utiliser les poids de classes")
    print("   • Surveiller les métriques par classe")
else:
    print("   ✅ Dataset relativement équilibré")

print("\n🏗️ ARCHITECTURE DE MODÈLE RECOMMANDÉE")
print("   Pour la classification d'images médicales :")
print("   • Transfer Learning avec des modèles pré-entraînés")
print("   • ResNet50/101 ou EfficientNet (recommandés pour l'imagerie médicale)")
print("   • Fine-tuning des dernières couches")
print("   • Dropout et régularisation pour éviter le surapprentissage")

print("\n📊 MÉTRIQUES D'ÉVALUATION RECOMMANDÉES")
print("   Pour un contexte médical :")
print("   • Sensibilité (Recall) - Crucial pour détecter la pneumonie")
print("   • Spécificité - Important pour éviter les faux positifs")
print("   • F1-Score - Équilibre entre précision et rappel")
print("   • AUC-ROC - Performance globale du modèle")
print("   • Matrice de confusion détaillée")

print("\n🔄 AUGMENTATION DE DONNÉES RECOMMANDÉE")
print("   Techniques adaptées aux radiographies :")
print("   • Rotation légère (±10-15°)")
print("   • Translation horizontale/verticale")
print("   • Zoom léger")
print("   • Ajustement de contraste et luminosité")
print("   • ⚠️ Éviter : flip horizontal (anatomie)")

print("\n⚙️ CONFIGURATION D'ENTRAÎNEMENT")
print(f"   • Taille d'image recommandée : {IMAGE_SIZE}x{IMAGE_SIZE}")
print(f"   • Batch size : {BATCH_SIZE}")
print(f"   • Learning rate initial : {LEARNING_RATE}")
print(f"   • Nombre d'époques max : {EPOCHS}")
print(f"   • Early stopping patience : {PATIENCE}")

print("\n📋 VALIDATION ET TEST")

# Analyse de la répartition train/val/test
train_total = sum(stats['par_subset']['train'].values()) if 'train' in stats['par_subset'] else 0
val_total = sum(stats['par_subset']['val'].values()) if 'val' in stats['par_subset'] else 0
test_total = sum(stats['par_subset']['test'].values()) if 'test' in stats['par_subset'] else 0

total_images = train_total + val_total + test_total

if total_images > 0:
    train_pct = (train_total / total_images) * 100
    val_pct = (val_total / total_images) * 100
    test_pct = (test_total / total_images) * 100
    
    print(f"   Répartition actuelle :")
    print(f"   • Train : {train_total:,} ({train_pct:.1f}%)")
    print(f"   • Validation : {val_total:,} ({val_pct:.1f}%)")
    print(f"   • Test : {test_total:,} ({test_pct:.1f}%)")
    
    if val_pct < 10:
        print("   ⚠️ ATTENTION : Ensemble de validation très petit !")
        print("   Recommandation : Redistribuer les données (70/15/15)")
    
    if test_pct < 10:
        print("   ⚠️ ATTENTION : Ensemble de test petit !")
        print("   Recommandation : Augmenter la taille du test set")

print("\n🎯 OBJECTIFS DE PERFORMANCE")
print("   Pour un modèle de détection de pneumonie :")
print("   • Sensibilité cible : > 90% (détecter la pneumonie)")
print("   • Spécificité cible : > 85% (éviter faux positifs)")
print("   • F1-Score cible : > 0.87")
print("   • AUC-ROC cible : > 0.90")

## 7. Conclusions

### Résumé de l'analyse

In [ ]:
# Génération du résumé final
print("📋 RÉSUMÉ DE L'ANALYSE")
print("=" * 50)

print("🔍 DATASET ANALYSÉ :")
print(f"   • Source : Kaggle - Paul Mooney")
print(f"   • Total d'images : {stats['total_images']:,}")
print(f"   • Classes : {', '.join(CLASSES)}")
print(f"   • Sous-ensembles : {', '.join(SUBSETS)}")

print("⚖️ DÉSÉQUILIBRE DES CLASSES :")
print(f"   • Ratio : {ratio_desequilibre:.2f}:1")
print(f"   • Classe majoritaire : {max(total_par_classe, key=total_par_classe.get)}")
print(f"   • Classe minoritaire : {min(total_par_classe, key=total_par_classe.get)}")

print("🖼️ PROPRIÉTÉS DES IMAGES :")
print(f"   • Dimensions moyennes : {proprietes['largeur_moyenne']:.0f}x{proprietes['hauteur_moyenne']:.0f}px")
print(f"   • Taille moyenne : {proprietes['taille_moyenne_mb']:.2f} MB")
print(f"   • Format principal : {max(proprietes['formats'], key=proprietes['formats'].get)}")

print("🚨 PROBLÈMES IDENTIFIÉS :")
if val_pct < 10:
    print("   • Ensemble de validation très petit")
if ratio_desequilibre > 2.0:
    print("   • Déséquilibre significatif des classes")
if len(set(proprietes['dimensions'])) > 10:
    print("   • Dimensions d'images variables")

print("✅ RECOMMANDATIONS CLÉS :")
print("   • Utiliser Transfer Learning (ResNet50/EfficientNet)")
print("   • Appliquer les poids de classes calculés")
print("   • Augmentation de données ciblée")
print("   • Métriques médicales (Sensibilité, Spécificité)")
print("   • Redistribution train/val/test si nécessaire")

print("🎯 PROCHAINES ÉTAPES :")
print("   1. Preprocessing et normalisation des images")
print("   2. Implémentation du modèle avec Transfer Learning")
print("   3. Entraînement avec validation croisée")
print("   4. Évaluation avec métriques médicales")
print("   5. Optimisation et déploiement")

print("\n🎉 Analyse terminée avec succès !")
print("📊 Rapport détaillé sauvegardé dans outputs/")
print("📈 Visualisations générées et affichées")

### Sauvegarde du rapport d'analyse

In [ ]:
# Sauvegarde du rapport complet
print("💾 Sauvegarde du rapport d'analyse...")

# Création du rapport complet
rapport_complet = {
    'metadata': {
        'dataset_source': 'Kaggle - Paul Mooney',
        'dataset_url': 'https://www.kaggle.com/datasets/paultimothymooney/chest-xray-pneumonia',
        'analyse_date': pd.Timestamp.now().strftime('%Y-%m-%d %H:%M:%S'),
        'auteur': 'Dady Akrou Cyrille'
    },
    'statistiques_dataset': stats,
    'proprietes_images': proprietes,
    'poids_classes': poids_classes,
    'analyse_desequilibre': {
        'ratio': ratio_desequilibre,
        'classe_majoritaire': max(total_par_classe, key=total_par_classe.get),
        'classe_minoritaire': min(total_par_classe, key=total_par_classe.get)
    },
    'recommandations': {
        'architecture': 'Transfer Learning avec ResNet50/EfficientNet',
        'gestion_desequilibre': 'Poids de classes + SMOTE',
        'metriques': ['Sensibilité', 'Spécificité', 'F1-Score', 'AUC-ROC'],
        'augmentation_donnees': ['Rotation', 'Translation', 'Zoom', 'Contraste']
    }
}

# Sauvegarde en JSON
os.makedirs('outputs', exist_ok=True)
with open('outputs/rapport_analyse_notebook.json', 'w', encoding='utf-8') as f:
    json.dump(rapport_complet, f, indent=2, ensure_ascii=False)

print("✅ Rapport sauvegardé : outputs/rapport_analyse_notebook.json")
print("\n🎯 Analyse complète terminée !")
print("📚 Ce notebook peut être utilisé comme référence pour le développement du modèle.")

---

## 📚 Références et Crédits

- **Dataset :** [Chest X-Ray Images (Pneumonia)](https://www.kaggle.com/datasets/paultimothymooney/chest-xray-pneumonia)
- **Créateur du dataset :** Paul Mooney
- **Plateforme :** Kaggle
- **Auteur de l'analyse :** Dady Akrou Cyrille
- **Date :** 2024

---

*Ce notebook présente une analyse complète du dataset de radiographies thoraciques pour la détection de pneumonie. Toutes les visualisations et recommandations sont basées sur l'analyse statistique des données.*